In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
awards_filepath = "src/files/220k_awards_by_directors.csv"
movies_filepath     = "src/files/AllMoviesDetailsCleaned.csv"
cast_filepath       = "src/files/AllMoviesCastingRaw.csv"
acclaimed_filepath = "src/files/900_acclaimed_directors_awards/900_acclaimed_directors_awards.csv"

awards_df = pd.read_csv(awards_filepath, delimiter=",", na_values='none')
movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')
cast_df = pd.read_csv(cast_filepath, delimiter=";", na_values='none')
acclaimed_df = pd.read_csv(acclaimed_filepath, delimiter=";", na_values='none')



C:\Users\rober\AppData\Local\Temp\ipykernel_32532\1185451972.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')


In [3]:
# Standardize capitalization and spacing across directors

def standardize(df, dir_col):
    df[dir_col] = df[dir_col].str.strip().str.title().str.replace('[\W]+', ' ', regex=True)
    
standardize(awards_df, 'director_name')
standardize(cast_df, 'director_name')
standardize(acclaimed_df, 'name')

In [4]:
awards_df.head()

,director_name,ceremony,year,category,outcome,original_language
0,Aki Kaurismäki,ACCEC Awards,2004.0,ACCEC Award,Won,fi
1,Aki Kaurismäki,Alliance of Women Film Journalists,2012.0,EDA Award,Nominated,fi
2,Aki Kaurismäki,"Amanda Awards, Norway",1999.0,Amanda,Nominated,fi
3,Aki Kaurismäki,Argentinean Film Critics Association Awards,2013.0,Silver Condor,Nominated,fi
4,Aki Kaurismäki,Argentinean Film Critics Association Awards,2004.0,Silver Condor,Nominated,fi


In [5]:
print(movies_df.columns)
movies_df.head()

Index(['id', 'budget', 'genres', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'production_companies_number',
       'production_countries_number', 'spoken_languages_number'],
      dtype='object')


,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5


In [6]:
cast_df.head()

,id,actor1_name,actor1_gender,actor2_name,actor2_gender,actor3_name,actor3_gender,actor4_name,actor4_gender,actor5_name,actor5_gender,actor_number,director_name,director_gender,director_number,producer_name,producer_number,screeplay_name,editor_name
0,2,Turo Pajala,0,Susanna Haavisto,0.0,Matti Pellonpää,2,Eetu Hilkamo,0,NaN,0,4,Aki Kaurismäki,0.0,1,NaN,0,Aki Kaurismäki,Raija Talvio
1,3,Matti Pellonpää,2,Kati Outinen,1.0,Sakari Kuosmanen,2,Esko Nikkari,2,Kylli Köngäs,0,7,Aki Kaurismäki,0.0,1,Mika Kaurismäki,1,Aki Kaurismäki,Raija Talvio
2,5,Tim Roth,2,Antonio Banderas,2.0,Jennifer Beals,1,Madonna,1,Marisa Tomei,1,24,Allison Anders,1.0,4,Lawrence Bender,1,NaN,Margaret Goodspeed
3,6,Emilio Estevez,2,Cuba Gooding Jr.,2.0,Denis Leary,2,Jeremy Piven,2,Peter Greene,2,15,Stephen Hopkins,2.0,1,Gene Levy,1,Lewis Colick,Tim Wellburn
4,8,NaN,0,NaN,0.0,NaN,0,NaN,0,NaN,0,0,Timo Novotny,0.0,1,Timo Novotny,2,Michael Glawogger,Timo Novotny


In [7]:
acclaimed_df.head()

,name,tmdbID,imdbID,Total awards,Academy Fellowship,Academy Fellowship_won,Academy Fellowship_nominated,Gold Hugo,Gold Hugo_won,Gold Hugo_nominated,...,Orpheus Award_nominated,Arte France Cinéma Award,Arte France Cinéma Award_won,Arte France Cinéma Award_nominated,Citizens Award,Citizens Award_won,Citizens Award_nominated,Cowboy Award,Cowboy Award_won,Cowboy Award_nominated
0,Ingmar Bergman,6648,nm0000005,101,1,1,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
1,Marlon Brando,3084,nm0000008,58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Federico Fellini,4415,nm0000019,90,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Alfred Hitchcock,2636,nm0000033,53,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Buster Keaton,8635,nm0000036,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Make the directory "output" if it isn't present
import os
if not os.path.exists("output"):
    os.mkdir("output")

In [9]:
def processPerson(row, person_data, rel_data, columns, role, extra_label):
    # Must be within the bounds
    if len(columns) < 1 or len(columns) > 2:
        return
    
    item = row[columns]
    
    # Get person info
    name = item[0]
    
    # Skip if NaN
    if name is np.nan:
        return
    
    if len(columns) == 2:
        if item[1] == 1:
            gender = "f"
        elif item[1] == 2:
            gender = "m"
        else:
            gender = "na"
    else:
        gender = "na"
        
    # Populate person info
    if name in person_data:
        item_new = person_data[name]
        # Process gender
        if gender == "f" or gender == "m":
            item_new[1] = gender
        # Process labels
        if extra_label not in item_new[2]:
            item_new[2] = item_new[2] + "|" + extra_label
    else:
        item_new = [name, gender, "Person" + "|" + extra_label]
            
    # Movie relationship
    movieID = row["id"]
    person_data[name] = item_new
    rel_data.append([name,role,movieID])
    
def processDirectors(person_data, dir_df, dir_file):
    
    
    dir_df['gender'] = dir_df['name'].map(lambda name: person_data[name][1] if name in person_data else 'na')
    dir_df['label']  = dir_df['name'].map(lambda name: person_data[name][2] if name in person_data else 'Person|Director')
    # Remove from persons data
    for name in dir_df['name']:
        if name in person_data:
            del person_data[name]
    
    
    dir_header = ["name:ID(Person-ID)", "gender", "tmdbID", "imdbID", "totalAwardsWonNom:int", ":LABEL"]
    dir_df[["name","gender","tmdbID","imdbID","Total awards","label"]].to_csv(dir_file, sep=";", index=False,
                                                                             lineterminator="", encoding="utf-8", header=dir_header)
        
def processCast(df, dir_df, person_file, dir_file, roles_file):
    
    persons_header = ["name:ID(Person-ID)", "gender", ":LABEL"]
    roles_header = [":START_ID(Person-ID)",":TYPE", ":END_ID(Movie-ID)"]
    
    # Set the director name as the index
    df["dir_index"] = df["director_name"]
    df.set_index("dir_index",inplace=True)
    
    col_actor1 = ['actor1_name', 'actor1_gender']
    col_actor2 = ['actor2_name', 'actor2_gender']
    col_actor3 = ['actor3_name', 'actor3_gender'] 
    col_actor4 = ['actor4_name', 'actor4_gender']
    col_actor5 = ['actor5_name', 'actor5_gender']
    col_director = ['director_name', 'director_gender']
    col_producer = ['producer_name']
    col_screenplay = ['screeplay_name']
    col_editor = ['editor_name']
    
    
    person_data = {}
    roles_data = []
    
    dir_names = dir_df['name']
    
    names_filtered = [name for name in dir_names if name in df.index]
    
    print("Names given:",len(dir_names), "; Names used:", len(names_filtered))
    
    df_filtered = df.loc[names_filtered]
    
    print("Dataframe filtered, movies given:", len(df), "; Movies used:", len(df_filtered))
    
    
    for i, row in df_filtered.iterrows():
        processPerson(row, person_data, roles_data, col_actor1, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor2, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor3, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor4, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor5, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_director, "DIRECTED", "Director")
        processPerson(row, person_data, roles_data, col_producer, "PRODUCED", "Producer")
        processPerson(row, person_data, roles_data, col_screenplay, "SCREENPLAYED", "Screenwriter")

    # Write directors file
    processDirectors(person_data, dir_df, dir_file)

    # write persons file 
    with open(person_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(persons_header)
        for row in person_data.values():
            writer.writerow(row)
    # wirte role relation file 
    with open(roles_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(roles_header)
        for row in roles_data:
            writer.writerow(row)
    return df_filtered['id']

In [10]:
movie_ids = processCast(cast_df, acclaimed_df, "output/person.csv", "output/directors.csv", "output/role.csv")

Names given: 893 ; Names used: 815
Dataframe filtered, movies given: 329044 ; Movies used: 14496


In [11]:
def processMovies(df, movie_ids, movie_file, company_file, company_produced_file):
    movie_header = ["movieID:ID(Movie-ID)", "title", "genres:string[]", "overview", "language", "rating:float", "votes:int", "budget:int", "revenue:int", "runtime:int", ":LABEL"]
    cur_df = df
    cur_df["label"]= "Movie"

    cur_df['spoken_languages'].replace("No Language", "",inplace=True)
    
    cur_df["id_index"] = cur_df["id"]
    cur_df.set_index("id_index", inplace=True)
    
    filter_df = cur_df.loc[movie_ids].copy()
    
    
    # write movies file 
    filter_df[["id", "title", "genres", "overview", "original_language",  "vote_average", "vote_count", "budget", "revenue", "runtime", "label"]].to_csv(movie_file, sep=";", index=False,
                                                                                                  lineterminator="", encoding="utf-8", header=movie_header)

    
    filter_df["company_label"] = "Company"
    filter_df["company_rel"] = "COMPANY_PRODUCED|PRODUCED"
    
    company_header = ["name:ID(Company-ID)", ":LABEL"]
    roles_header = [":START_ID(Company-ID)",":TYPE", "productionCountry", ":END_ID(Movie-ID)"]
    
    # Write company node file
     # get nodes
    companies = []
    for company in filter_df["production_companies"].unique():
        companies.append([company, "Company"])
            
     # write nodes file 
    with open(company_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(company_header)
        for row in companies:
            writer.writerow(row)
            
    # Write company relationship file
    filter_df[["production_companies","company_rel","production_countries", "id"]].to_csv(company_produced_file, sep=";", index=False,
                                  lineterminator="", encoding="utf-8", header=roles_header)
    

In [12]:
processMovies(movies_df, movie_ids, "output/movie.csv", "output/company.csv", "output/company_role.csv")

In [13]:
def processAwards(df, directors, node_file, relation_file):
    awards_header = ["name:ID(Award-ID)", ":LABEL"]
    relation_header = [":START_ID(Person-ID)",":TYPE", "years:int[]", ":END_ID(Award-ID)"]
    cur_df = df
    cur_df['name_clean'] = cur_df['director_name']
    
    
    cur_df['director_id'] = cur_df['name_clean']
    cur_df.set_index('director_id', inplace=True)
    
    director_names = directors['name']
    
    names_filtered = [name for name in director_names if name in cur_df.index]
    
    filter_df = cur_df.loc[names_filtered]
    
    print("Names given:",len(director_names), "; Names used:", len(names_filtered))
    
    print("Dataframe filtered, awards given:", len(df), "; Movies used:", len(filter_df))
    
    
    cur_df2 = filter_df[['name_clean','category','outcome','year']].groupby(['name_clean','category','outcome'])['year'].unique().reset_index()
    cur_df2['yearstr'] = cur_df2.year.apply(lambda x : "|".join("%4.0f" % y for y in x))
    
    cur_df3 = cur_df2[['name_clean','outcome','yearstr','category']]
    
    
    
    awards = []
    
    # get nodes
    for award in filter_df["category"].unique():
        awards.append([award, "Award"])
            
    # write nodes file 
    with open(node_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(awards_header)
        for row in awards:
            writer.writerow(row)
    # wirte relation file 
    cur_df3.to_csv(relation_file, sep=";", index=False,
                  lineterminator="", encoding="utf-8", header=relation_header)


In [14]:
processAwards(awards_df, acclaimed_df, "output/award.csv", "output/awarded.csv")

Names given: 893 ; Names used: 773
Dataframe filtered, awards given: 225675 ; Movies used: 27400


In [15]:
# Copy or move .csv products to the database's import folder.
# Open the database terminal.
# run neo4j-admin to load data to neo4j
# .\bin\neo4j-admin.bat import --force --multiline-fields=true --delimiter=";" --array-delimiter="|" --database=final --nodes=.\import\award.csv --nodes=.\import\movie.csv --nodes=.\import\person.csv --nodes=.\import\director.csv --relationships=.\import\awarded.csv --relationships=.\import\role.csv